In [1]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
import pdfplumber
import spacy
import re

# ETL (Extract, Transform, Load)

## Extracción (Extract)

In [2]:
# Extraer el texto del pdf: 
texto_extraido = ""
with pdfplumber.open("anexo-15.pdf") as pdf:
    for page in pdf.pages:
        texto = page.extract_text()
        texto_extraido = (texto_extraido + texto + "\n\n") # Convertimos el pdf en un string separando cada pagina con 2 saltos de línea              

## Transformación (Transform)

#### Eliminar los pies de página

In [3]:
# Eliminamos las 5 últimas líneas de todas las páginas:

# Dividir el texto en líneas
lineas = texto_extraido.split("\n")

# Identificar posiciones de final de página:
finales_de_pagina = []
for i in range(len(lineas)):                
    if lineas[i]== "" :
        finales_de_pagina.append(i)
        
# Marcar las 5 últimas líneas de cada página:
lineas_a_eliminar = []
for linea in finales_de_pagina:
    for i in range(linea - 5, linea):
        lineas_a_eliminar.append(i)

# Filtrar el texto por las líneas que queremos mantener:
lineas_filtradas = [linea for i, linea in enumerate(lineas) if i not in lineas_a_eliminar]

# Reconstruir el texto limpio
texto_limpio = "\n".join(lineas_filtradas)

In [4]:
#Eliminamos 2 líneas de pie de página más que no aparecen en todas las páginas:

from collections import Counter

# Dividir el texto en líneas
lineas = texto_limpio.split("\n")

# Contar frecuencia de cada línea
frecuencias = Counter(lineas)

# Filtrar líneas cuya frecuencia (18) corresponde a los pies de página:
umbral = 18  
lineas_filtradas = [linea for linea in lineas if frecuencias[linea] != umbral]

# Reconstruir el texto limpio
texto = "\n".join(lineas_filtradas)

#### POS y lemma

- POS (Parts of Speech) tagging: Etiquetado de categoría gramatical 

- Lematización: Reducción de las palabras a su forma base o lema

In [5]:
# Procesamiento de lenguaje natural con spacy: 
nlp = spacy.load("es_core_news_lg")

In [6]:
# Obtención de POS y lema para cada elemento del texto:
datos = []
doc = nlp(texto)
for token in doc:
    datos.append({
        "Texto": token.text,
        "Lema": token.lemma_,
        "POS": token.pos_
    })
df = pd.DataFrame(datos)
df 

,Texto,Lema,POS
0,DEPARTAMENTO,DEPARTAMENTO,PROPN
1,DE,DE,ADP
2,\n,\n,SPACE
3,INSPECCIÓN,inspección,NOUN
4,Y,Y,CCONJ
...,...,...,...
8677,la,el,DET
8678,\n,\n,SPACE
8679,venta,venta,NOUN
8680,.,.,PUNCT


In [7]:
# ¿Cuántas categorías gramaticales hay?
df["POS"].value_counts()

POS
NOUN     2015
ADP      1436
DET      1215
PUNCT     867
SPACE     616
ADJ       575
VERB      437
AUX       316
CCONJ     309
PROPN     285
PRON      224
SCONJ     137
NUM       128
ADV       121
PART        1
Name: count, dtype: int64

In [8]:
# Normalizar formato (poner los lemas en minúscula): 
df["Lema"]= df["Lema"].str.lower()

In [9]:
# ¿Cuáles son los sustantivos que más se repiten?
df["Lema"][df["POS"]== 'NOUN'].value_counts().head(15)

Lema
proceso          115
validación       108
producto          78
equipo            47
calidad           43
producción        36
limpieza          35
cualificación     34
lote              32
instalación       30
enfoque           30
sistema           29
verificación      29
control           28
cambio            28
Name: count, dtype: int64

In [10]:
# ¿Cuáles son los verbos que más aparecen?
df["Lema"][df["POS"].isin(['AUX','VERB'])].value_counts().head(15)

Lema
deber         141
ser            68
poder          52
estar          29
haber          24
utilizar       15
confirmar      15
incluir        14
considerar     11
cumplir        11
realizar       10
demostrar      10
tener          10
definir         9
evaluar         8
Name: count, dtype: int64

In [11]:
# ¿Cuáles son los sustantivos que más se repiten?
df["Lema"][df["POS"]== 'ADJ'].value_counts().head(15)

Lema
documentado    14
utilizado      11
aprobado       11
justificado    10
definido       10
apropiado      10
continuo       10
siguiente      10
tradicional     9
necesario       9
aceptable       9
crítico         9
validado        8
aplicable       7
importante      7
Name: count, dtype: int64

## Carga (Load)

In [12]:
#Guardar el DataFrame en un archivo CSV: 
df.to_csv('anexo-15.csv')

# Aprendizajes e intentos 

#### Decidir si trabajar con el texto completo o por fragmentos (epígrafes)

- Finalmente he optado por trabajar con el texto completo al no encontrar diferencias significativas en los resultados y para preservar mejor el contexto.
- La división por epígrafes no ha sido del todo satisfactoria a pesar de usar regex para afinar el filtro. 

In [13]:
# PDF por páginas
#Abrimos el pdf y extraemos el texto de cada página añadiéndolo a una lista que transformamos en df:
paginas = []
with pdfplumber.open("anexo-15.pdf") as pdf: 
    for p in pdf.pages:
        pagina= p.extract_text() 
        paginas.append(pagina)

# PDF por epígrafes
#Dividimos cada página de la lista en sublistas de epígrafes usando expresiones regulares y creando una nueva lista que pasamos a df:
import re
epigrafes_por_pagina = []

for p in paginas:
    epigrafes = re.split(r"(?<=\.|:|[A-Z]|\))\n(?=[A-Z][a-z]|\d+)", p)
    epigrafes_por_pagina.append(epigrafes)
df_epigrafes = pd.DataFrame(epigrafes_por_pagina).T
df_epigrafes

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,DEPARTAMENTO DE\nINSPECCIÓN Y CONTROL DE\nMEDI...,Principio\nEste anexo describe los principios ...,1.4. Los elementos clave del programa de cuali...,2.2. Todos los documentos generados durante la...,"3. ETAPAS DE CUALIFICACIÓN DE EQUIPOS, INSTALA...","3.9. La IQ debe incluir, entre otros aspectos,...",ii. Las pruebas deben cubrir el rango de funci...,5.4. La validación del proceso de nuevos produ...,"5.13. Es especialmente importante que, con el ...",5.21. Debe prepararse un protocolo de validaci...,consecución del producto. Esto también debe in...,6. VERIFICACIÓN DEL TRANSPORTE,"8.3. Para mitigar cualquier riesgo de fallo, d...","ejemplo, los operarios, el nivel de detalle en...",el resultado. Debe demostrarse que es posible ...,12. GLOSARIO\nA continuación se dan las defini...,La verificación documentada de que las instala...,Especificación de requisitos de usuario\nEl co...,Verificación de la limpieza\nLa obtención de e...
1,Guía de Normas de Correcta Fabricación de\nMed...,General\nDebe emplearse un enfoque de la gesti...,1.5. El PMV o documento equivalente debe defin...,"2.3. En los proyectos de validación complejos,...",3.1. En las actividades de cualificación deben...,Cualificación del funcionamiento (del inglés O...,4. RECUALIFICACIÓN,5.5. En la validación del proceso de productos...,5.14. En el caso de que se vayan a liberar al ...,5.22. Los protocolos de validación del proceso...,5.25. Los principios generales establecidos en...,"6.1. Los productos terminados, los medicamento...",9. VALIDACIÓN DE MÉTODOS ANALÍTICOS,10.6. Los límites para el arrastre de residuos...,10.13. Para demostrar que el método de limpiez...,Agentes simulados\nUn material cuyas caracterí...,Cualificación del Funcionamiento (OQ),Estado de control\nUna condición en la cual el...,Validación concurrente\nValidación efectuada e...
2,Estado del documento: revisado.,1. ORGANIZACIÓN Y PLANIFICACIÓN DE LA CUALIFIC...,1.6. Cuando se trate de proyectos grandes y co...,2.4. Se deben elaborar protocolos de validació...,Especificación de los requisitos de usuario (d...,3.10 La OQ normalmente se realiza después de l...,"4.1. Se deben evaluar los equipos, las instala...",5.6. Para la transferencia a otra planta de pr...,5.15. Para la validación del Proceso de Medica...,Verificación continua del proceso\n5.23. En el...,Enfoque híbrido\n5.26. Puede aplicarse un enfo...,6.2. Se reconoce que la verificación del trans...,9.1. Todos los métodos de análisis usados en l...,10.6.1. Es conocido que las macromoléculas ter...,10.14. Si un proceso de limpieza no es efectiv...,Atributo de Calidad Crítico (CQA),La verificación documentada de que las instala...,Estrategia de Control\nUn conjunto planificado...,Verificación continua del proceso\nUn enfoque ...
3,Razones de los cambios: desde la publicación d...,1.1. Todas las actividades de cualificación y ...,1.7. Se debe utilizar un enfoque de gestión de...,2.5. Los documentos de cualificación se pueden...,"3.2. La especificación de los equipos, las ins...","3.11 La OQ debe incluir, entre otros aspectos,...",4.2. Cuando se requiera una recualificación y ...,5.7. La validación del proceso debe establecer...,Validación concurrente del proceso\n5.16. En c...,5.24. Debe definirse el método por el cual se ...,5.27. Este enfoque también puede usarse para c...,Durante la verificación del transporte deben c...,9.2. Cuando se lleven a cabo análisis microbio...,10.6.2. Si no fuera posible realizar el ensayo...,10.15. Cuando se realice una limpieza manual d...,"Una propiedad o característica física, química...",Cualificación de la ejecución del Proceso (PQ),Gestión del conocimiento\nUn enfoque sistemáti...,Verificación en curso del proceso (también con...
4,Fecha de entrada en vigor: 1 de octubre de 2015.,1.2. Las actividades de cualificación y de val...,1.8. Deben incorporarse los controles adecuado...,2.6. Cuando los protocolos de validación y dem...,Cualificación del diseño (del ingl

#### NER (Named Entity Recognition): reconocimiento de entidades con nombre

- El reconocimiento de entidades con nombre en el Anexo 15 ha sido limitado, probablemente porque el modelo está pensado para lenguaje general y no específicamente para textos técnicos.
- Valoro la posibilidad de encontrar un modelo en Hugging Face que se adapte mejor a este tipo de documentos. 
- El modelo parece más afinado cuando se aplica sobre el texto completo que sobre fragmentos como los epígrafes.

In [14]:
# Sobre texto completo: 
entidades = []
doc = nlp(texto)
for ent in doc.ents:
    entidades.append({
        "Entidad": ent.text,
        "Etiqueta": ent.label_
    })
df_entidades = pd.DataFrame(entidades).reset_index()
df_entidades 

,index,Entidad,Etiqueta
0,0,DEPARTAMENTO DE\nINSPECCIÓN Y CONTROL DE,ORG
1,1,MEDICAMENTOS\nGuía de Normas de Correcta Fabri...,MISC
2,2,Veterinario\nAnexo 15: Cualificación,LOC
3,3,Directiva,ORG
4,4,CE,ORG
...,...,...,...
231,231,Validación de la limpieza\nLa validación de la...,MISC
232,232,Validación,MISC
233,233,Validación,MISC
234,234,Validación,MISC


In [15]:
# Sobre epígrafes:
epigrafes= []
for pagina in epigrafes_por_pagina:
    for epigrafe in pagina:
        for ent in nlp(epigrafe).ents:
            epigrafes.append({
                "Texto":ent.text,
                "Entidad":ent.label_
            })
pd.DataFrame(epigrafes).reset_index()

,index,Texto,Entidad
0,0,DEPARTAMENTO DE\nINSPECCIÓN Y CONTROL DE,ORG
1,1,MEDICAMENTOS,ORG
2,2,Medicamentos de Uso Humano,MISC
3,3,Veterinario\nAnexo 15: Cualificación,LOC
4,4,Directiva,ORG
...,...,...,...
333,333,Validación,MISC
334,334,MINISTERIO DE SANIDAD,LOC
335,335,SERVICIOS SOCIALES\nE IGUALDAD,MISC
336,336,Página 19 de 19\n,MISC


#### Análisis del texto por palabras:

- He observado que es preferible trabajar con el texto completo en lugar de fragmentado, ya que así se conserva mejor el contexto y las relaciones entre palabras.
- La lematización depende de qué POS se le haya asignado a cada palabra y para un correcto etiquetaje de POS es necesario el contexto.

#### Selección del modelo de spaCy:

- He usado el modelo en español `es_core_news_lg` al ser más completo que los modelos `sm` y `md`.

#### Separar el df por tipo de palabra

- La aplicación de POS tagging no ha sido del todo precisa, hay números que se etiquetan como NOUN. 

#### Simplificación del análisis

- Inicialmente establecí funciones para filtrar por tipos de palabras (VERB, NOUN, ADJ, ADV) y analizar cada categoría por separado.
- Finalmente me he decantado por un análisis más sencillo, mediante máscaras booleanas como filtros en lugar de funciones.

In [ ]:
# Función que detecta si una celda contiene algún verbo
def contiene_verbo(texto):
    doc = nlp(str(texto))  # convierte a string por si hay valores nulos
    return any(tok.pos_ == "VERB" for tok in doc)
df_verbos3 = df_sn.copy()
df_verbos3 = df_verbos3['palabras'][df_verbos3['palabras'].apply(contiene_verbo)]
df_verbos3

In [ ]:
#Sustantivos
def contiene_sustantivo(texto):
    doc = nlp(str(texto))  # Convierte a string por si hay valores nulos
    return any(tok.pos_ == "NOUN" for tok in doc)
df_sust = df_sn.copy()
df_sust = df_sust['palabras'][df_sust['palabras'].apply(contiene_sustantivo)]
df_sust

In [ ]:
#Adjetivos
def contiene_adjetivo(texto):
    doc = nlp(str(texto))  # Convierte a string por si hay valores nulos
    return any(tok.pos_ == "ADJ" for tok in doc)
df_adj =df_sn.copy()
df_adj = df_adj['palabras'][df_adj['palabras'].apply(contiene_adjetivo)]
df_adj

In [ ]:
#Otros
def sin_sust_adj_verbos(texto):
    doc = nlp(str(texto))  # Convertimos a string por si hay NaNs
    return not any(tok.pos_ in ["NOUN", "ADJ", "VERB"] for tok in doc)
df_otro = df_sn.copy()
df_otro = df_otro['palabras'][df_otro['palabras'].apply(sin_sust_adj_verbos)]
df_otro

In [ ]:
# Combinar los df
dfs = [df_numeros, df_verbos3, df_sust, df_adj, df_otro]
df_comb = pd.concat([df.reset_index(drop=True) for df in dfs], axis=1)
df_comb.columns = ['numeros','verbos', 'sustantivos', 'adjetivos', 'otros']
df_comb

In [ ]:
# Ordenar el contenido alfabéticamente, ignorando NaN y añadiéndolos al final:
df_ordenado = pd.DataFrame({
    col: sorted(df_comb[col].dropna()) + [np.nan] * df_comb[col].isna().sum()
    for col in df_comb.columns
})
df_ordenado